In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
import git
from pathlib import Path

# get root of git repo
prefix = Path(git.Repo(".", search_parent_directories=True).working_tree_dir)

# azure ml settings
experiment_name = "sklearn-diabetes-mlproject-example"

In [ ]:
import mlflow

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

In [ ]:
from azureml.core import Run

run = next(ws.experiments[experiment_name].get_runs())
run

In [ ]:
run.id

In [ ]:
from azureml.core.webservice import AksWebservice

aks_config = AksWebservice.deploy_configuration(
    compute_target_name="aks-cpu-deploy",
    cpu_cores=2,
    memory_gb=5,
    tags={"data": "diabetes", "method": "sklearn"},
    description="Predict using webservice",
)

In [ ]:
import mlflow.azureml

webservice, azure_model = mlflow.azureml.deploy(
    model_uri=f"runs:/{run.id}/model",
    workspace=ws,
    deployment_config=aks_config,
    service_name="sklearn-diabetes",
    model_name="sklearn-diabetes-example",
)

In [ ]:
from sklearn import datasets

diabetes = datasets.load_diabetes()

import pandas as pd
import numpy as np

X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = [
    "age",
    "sex",
    "bmi",
    "bp",
    "s1",
    "s2",
    "s3",
    "s4",
    "s5",
    "s6",
    "progression",
]
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]

query_input = sample.to_json(orient="split")
query_input = eval(query_input)
query_input.pop("index", None)

In [ ]:
import requests
import json

headers = {"Content-Type": "application/json"}

response = requests.post(
    url=webservice.scoring_uri, data=json.dumps(query_input), headers=headers
)
print(response.text)

In [ ]:
webservice.delete()